# Data import for given station and given time period. Import also allows for parameter selection in output file.

In [14]:
from viresclient import SwarmRequest
import hapiclient
from hapiclient.util import pythonshell
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from chaosmagpy import load_CHAOS_matfile
from chaosmagpy.model_utils import synth_values
from chaosmagpy.data_utils import mjd2000 # Modified Julian date

import utils

The objective of this notebook is to download the full input data for a period of time and a given station. This notebook then exports the data to a csv file.

In [15]:
###Define station and time parameters, and OMNI parameters.

### Time parameters
start_date= 1998
end_date=2020

### Observatory parameters
observatory='NGK' 

### Download observatory metadata:
obsmeta = utils.fetch_ground_obs(observatory,'1998-01-01','1998-01-03')
latitude = obsmeta.attrs['Latitude_GEO']
longitude_deg = obsmeta.attrs['Longitude_GEO']
radius = obsmeta.attrs['Radius_GEO']


###OPTIONAL: input radius, latitude,longitude manually
#latitude=55.31
#longitude_deg=356.79 ##longitude is in degrees from Prime Meridian, i.e. 1 degree west is 359 degrees
#radius= 6371


Accessing INTERMAGNET and/or WDC data
Check usage terms at ftp://ftp.nerc-murchison.ac.uk/geomag/Swarm/AUX_OBS/minute/README



Processing:    0%|          |  [ Elapsed: 00:00, Remaining: ? ] [1/1] 

Downloading:   0%|          |  [ Elapsed: 00:00, Remaining: ? ] (0.174MB)

In [16]:
###Step 1; Get year data into correct time indexing format


start_date_str= f'{start_date}-01-01'
end_date_str=f'{end_date}-01-01'

year_range=pd.date_range(start_date_str, end_date_str, freq='1YS')

print('list of years to be downloaded:')
print(year_range)

list of years to be downloaded:
DatetimeIndex(['1998-01-01', '1999-01-01', '2000-01-01', '2001-01-01',
               '2002-01-01', '2003-01-01', '2004-01-01', '2005-01-01',
               '2006-01-01', '2007-01-01', '2008-01-01', '2009-01-01',
               '2010-01-01', '2011-01-01', '2012-01-01', '2013-01-01',
               '2014-01-01', '2015-01-01', '2016-01-01', '2017-01-01',
               '2018-01-01', '2019-01-01', '2020-01-01'],
              dtype='datetime64[ns]', freq='AS-JAN')


In [17]:
###Step 2; Download magnetic field model for given station.


CHAOS7 = utils.loadChaosModel(start_date,end_date,latitude,longitude_deg)
CHAOS7

###NOTE: This function assumes constant radius, which is not the case; radius varies according to latitude and height
###from sea level. Change radius= parameter (last parameter in function) to get more accurate.

###Further NOTE: this function does not include the last 24 hours of the period in question. Consider
###changing this if possible.

,B_radius,B_phi,B_theta,B_H
1998-01-01 00:00:00,-45003.735634,355.676092,-18613.382250,18616.780180
1998-01-01 00:01:00,-45003.735702,355.676163,-18613.382255,18616.780186
1998-01-01 00:02:00,-45003.735769,355.676235,-18613.382259,18616.780192
1998-01-01 00:03:00,-45003.735837,355.676306,-18613.382263,18616.780197
1998-01-01 00:04:00,-45003.735904,355.676378,-18613.382267,18616.780203
...,...,...,...,...
2020-12-30 23:56:00,-45848.121978,1365.227373,-18687.930509,18737.731786
2020-12-30 23:57:00,-45848.121978,1365.227373,-18687.930509,18737.731786
2020-12-30 23:58:00,-45848.121978,1365.227373,-18687.930509,18737.731786
2020-12-30 23:59:00,-45848.121978,1365.227373,-18687.930509,18737.731786


In [ ]:
###Step 3; loop over years and download observatory data
    
    
##Define start and end of download period for this year
start= start_date_str
end= end_date_str

print('Downloading Observatory data for',observatory,str(start)[:4],'to',str(end)[:4])

obs = utils.fetch_ground_obs(observatory, start, end)#Download obs data
obs = utils.manipulate_obs_data(obs,CHAOS7)#Manipulate obs data to remove background mag field


Accessing INTERMAGNET and/or WDC data
Check usage terms at ftp://ftp.nerc-murchison.ac.uk/geomag/Swarm/AUX_OBS/minute/README



          |  [ Elapsed: 00:00, Remaining: ?]  

Processing:    0%|          |  [ Elapsed: 00:00, Remaining: ? ] [1/3] 

Downloading:   0%|          |  [ Elapsed: 00:00, Remaining: ? ] (246.975MB)

In [ ]:
#print(events_tot['B_N'])

print(obs)

plt.plot(obs['Timestamp'])
plt.show()
plt.plot(obs['B_N'])
plt.show()

In [ ]:
###Step 4: get current directory and set PATH variable to where data will be saved.

import os

# Get the current working directory
current_path = os.getcwd()

# Print the current working directory
print("Current working directory:", current_path)

#define path to data folder where data will be placed. Print current path if you don't know what it is!
PATH = '/mnt/c/users/s1725621/Documents/PhDCode/Git/dBdtCorrelation/data/'

In [ ]:
###Step 5: save observatory data.

###Save to netcdf file
##obs.to_netcdf(path = PATH+observatory+str(start)[:4]+str(end)[:4]+'.nc')

###Save to csv file
# Convert xarray Dataset to pandas DataFrame
obf = obs.to_dataframe()

# Save the DataFrame to a CSV file
obf.to_csv(PATH+observatory+str(start)[:4]+str(end)[:4]+'.csv',index=True)

# Loop through observatories and download all into data folder.

In [ ]:
###Import list of observatories from csv.
df = pd.read_csv("data/ObsListimo.csv")
print(df['Code'])

In [ ]:
for i in df['Code']:
    try:
        ### Observatory parameters 
        i=str(i)
        
        ### Download observatory metadata:
        obsmetai = utils.fetch_ground_obs(i,'1998-01-01','1998-01-03')
        latitudei = obsmetai.attrs['Latitude_GEO']
        longitude_degi = obsmetai.attrs['Longitude_GEO']
        radiusi = obsmetai.attrs['Radius_GEO']
        
        ###Find model values for this station
        CHAOS7i = utils.loadChaosModel(start_date,end_date,latitudei,longitude_degi)
        
        ###Download data.
        obsi = utils.fetch_ground_obs(i, start, end)#Download obs data
        obsi = utils.manipulate_obs_data(obsi,CHAOS7i)#Manipulate obs data to remove background mag field
        
        ###Save to netcdf file
        ##obs.to_netcdf(path = PATH+observatory+str(start)[:4]+str(end)[:4]+'.nc')
        
        ###Save to csv file
        # Convert xarray Dataset to pandas DataFrame
        obfi = obsi.to_dataframe()
        
        # Save the DataFrame to a CSV file
        obfi.to_csv(PATH+i+str(start)[:4]+str(end)[:4]+'.csv',index=True)
    except:
        continue